In [1]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import csv

In [16]:
soup = BeautifulSoup (open("bovada20200703.html"), features="lxml")

In [17]:
games = soup.find_all("section", {"class": "coupon-content more-info"})

In [19]:
rows = []
for game in games:
    teams = game.find_all("span", {"class": "name"})
    verticals= game.find_all("sp-two-way-vertical", {"class": "market-type"})
    moneylines = verticals[1].find_all("span", {"class": "bet-price"})
    if len(moneylines) != 2:
        continue
    g = [teams[0].contents[0],teams[1].contents[0],moneylines[0].contents[0],moneylines[1].contents[0]]
    rows.append(g)

In [20]:
f = open('output/moneylines20200307.csv', 'w', newline = '')
w = csv.writer(f, delimiter = ',')
w.writerows(rows)
f.close()

In [21]:
#Cleans dirty US odds text and converts to payout relative to bet.
# -130 => 100/130 => 0.769
# +190 => 190/100 => 1.9

def getPayout(oddsText):
    oddsText = oddsText.strip()
    if oddsText[0]=="+":
        return float(oddsText[1:])/100
    elif oddsText[0]=="-":
        return 100/float(oddsText[1:])
    elif oddsText=="EVEN":
        return 100/110
    else:
        return None

In [22]:
import requests

#Calls FBS API with given team name and returns stats
def getTeamData(name):
    try:
        baseUrl="https://api.fastbreakstats.com/teams/"
        params="?q=current"
        url=baseUrl+name+params
    
        r = requests.get(url = url).json()[0]
        return r
    except IndexError:
        print("Team error: " + name)
        return None

In [44]:
import math

#Returns the confidence that the home team will win.
#These numbers come from Blake's built model.
def getHomeConfV1(away, home):
    t1coef_mult = float(away['KenPomVal']) * -0.04014262 + float(away['BPIval']) * -0.08951514 + float(away['SRS']) * -0.02285489
    t2coef_mult = float(home['KenPomVal']) *  0.09251246 + float(home['BPIval']) * 0.00297043  + float(home['SRS']) * 0.02869563
 
    coef_sum = (t1coef_mult + t2coef_mult)+.51305998
    eTOs = math.exp(coef_sum)
    conf = eTOs / (1+eTOs)
    return(conf)

def getHomeConfV2(away, home):
    t1coef_mult = float(away['KenPomVal']) * -0.14904683 + float(away['BPIval']) * 0.02408791 + float(away['SRS']) * -0.00285878
    t2coef_mult = float(home['KenPomVal']) *  0.14617775 + float(home['BPIval']) * -0.05714497+ float(home['SRS']) * 0.03016929
 
    coef_sum = (t1coef_mult + t2coef_mult)+0.53923323
    eTOs = math.exp(coef_sum)
    conf = eTOs / (1+eTOs)
    return(conf)

c = getTeamData('Cornell')
p = getTeamData('Princeton')

print('V1: ' + str(getHomeConfV1(c,p)))
print('V2: ' + str(getHomeConfV2(c,p)))

V1: 0.8692512138197256
V2: 0.8950650181191212


In [35]:
def fixName(name):
    if name=="Long Island University":
        return("LIU Brooklyn")
    if name=="Central Florida":
        return("UCF")
    if name=="St. Francis ":
        return("St. Francis (PA)")
    if name=="Mount St Mary's":
        return("Mt. St. Mary's")
    if name=="Texas San Antonio":
        return("UTSA")
    if name=="Florida International":
        return("FIU")
    if name=="Middle Tennessee St.":
        return("Middle Tennessee")
    if name=="Southern Mississippi":
        return("Southern Miss")
    if name=="Texas A&M Corpus":
        return("Texas A&M-CC")
    if name=="Miami Florida":
        return("Miami")
    if name=="CS Northridge":
        return("CSU Northridge")
    if name=="Cal Irvine":
        return("UC Irvine")
    return name

In [ ]:
import csv
import unicodedata
with open('output/moneylines20200307.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    output_rows=[]
    output_rows.append(['Game', 'awayPayout', 'homePayout', 'oldAwayConf', 'oldHomeConf', 'oldAwayEV', 'oldHomeEV', 'newAwayConf', 'newHomeConf', 'newAwayEV', 'newHomeEV'])
    for row in spamreader:
        #Calculate payouts for each team
        awayPayout=getPayout(row[2])
        homePayout=getPayout(row[3])
        
        #If odds aren't set on one skip the game
        if awayPayout==None or homePayout==None:
            continue
        
        # [Away, Home]
        teams=[row[0], row[1]]
        for i, team in enumerate(teams):
            
            #Removing leading and trailing spaces.
            #string.strip() seems to replace spaces with \xa0 spaces, so we undo that.
            teams[i]=team.strip()
            teams[i] = teams[i].replace(u'\xa0', u' ')
            
            #Get rid of ranks
            index=team.find("(")
            if index>=0:
                teams[i]=team[:index]
                
            teams[i]=teams[i].replace("State", "St.")
            teams[i]=fixName(teams[i])
        #Get team stats. Skip game if name error
        away=getTeamData(teams[0])
        home=getTeamData(teams[1])
        if away==None or home==None:
            continue

        #Calculate confidences
        homeConf=getHomeConfV1(away,home)
        awayConf=1-homeConf
        newHomeConf=getHomeConfV2(away,home)
        newAwayConf=1-newHomeConf
        
        #Calculate expected value of each bet
        awayEV=awayConf*awayPayout-homeConf
        homeEV=homeConf*homePayout-awayConf
        newAwayEV=newAwayConf*awayPayout-homeConf
        newHomeEV=newHomeConf*homePayout-awayConf
        
        r=[
            away['Team'][0] + " @ " + home['Team'][0],
            str(awayPayout), str(homePayout),
            str(awayConf), str(homeConf),
            str(awayEV),str(homeEV),
            str(newAwayConf), str(newHomeConf),
            str(newAwayEV), str(newHomeEV)
        ]
        output_rows.append(r)
        print(r)
        
    f = open('output/bets20200307.csv', 'w', newline = '')
    w = csv.writer(f, delimiter = ',')
    w.writerows(output_rows)
    f.close()

Team error: UL Lafayette
['Auburn @ Tennessee', '1.25', '0.6896551724137931', '0.47669650694487453', '0.5233034930551255', '0.0725671406259677', '-0.11579754621720179', '0.4528792663643946', '0.5471207336356054', '0.0427955899003678', '-0.09937186305825008']
["Marquette @ St. John's", '0.6451612903225806', '1.35', '0.5714390713592432', '0.4285609286407568', '-0.05989056002189019', '0.007118182305778542', '0.5823473301526678', '0.4176526698473321', '-0.05285297370355174', '-0.007607967065344834']
['Villanova @ Georgetown', '0.2702702702702703', '2.85', '0.6180440499732299', '0.3819559500267701', '-0.21491701760157278', '0.47053040760306486', '0.6454255329597014', '0.3545744670402986', '-0.20751661679441832', '0.3924931810916211']
['Wisconsin @ Indiana', '1.2', '0.7142857142857143', '0.44293994121626445', '0.5570600587837355', '-0.025532129324218222', '-0.04503989922788193', '0.45935051670557403', '0.540649483294426', '-0.0058394387370467316', '-0.05676173886310304']
['South Carolina @ V